### Import the necessary libraries

In [1]:
import yfinance as yf
import pandas as pd

### Load in the datasets

In [ ]:
name_path = 'SEC_data/2005_Financial_Analysis_Sample.xlsx'
stock_name = pd.read_excel(name_path)
print(stock_name.head())

### Transform the datasets

In [ ]:
# Select 'tsymbol' and 'sho_pilot' columns and remove duplicates
df_sho_pilot = stock_name[['tsymbol', 'sho_pilot']].drop_duplicates()
# Summarize the number of 1's and 0's for 'sho_pilot' column
summary_df = pd.DataFrame({
    'count_1': [df_sho_pilot['sho_pilot'].sum()],
    'count_0': [(df_sho_pilot['sho_pilot'] == 0).sum()]
})
# Display the summarized data frame
print(summary_df)

In [4]:
# price_path = 'SEC_data/GMW_LLS_codes/CRSP_Daily_AprilMay2005.dta'
# crsp_daily = pd.read_stata(price_path)
# # Filter rows where 'tsymbol' in crsp_daily is present in stock_name
# df_price = crsp_daily[crsp_daily['tsymbol'].isin(stock_name['tsymbol'])]
# df_price = df_price[['date', 'tsymbol', 'prc']]
# df_price = df_price.drop_duplicates(subset=['date', 'tsymbol'])
# df_price = df_price.pivot(index='date', columns='tsymbol', values='prc')

### Download data from Yahoo Finance

In [ ]:
# List of stock symbols (assuming stock_name['tsymbol'] is a pandas Series)
stock_symbols = stock_name['tsymbol'].tolist()

# Initialize an empty dictionary to store the price data
stock_data = {}

# Define the start and end date for the time range
start_date = "2005-01-01"
end_date = "2005-05-31"

# Initialize counters for success and failure
success_count = 0
failure_count = 0

# Initialize a list to store the availability (1 for success, 0 for failure)
availability = []

# Loop through each stock symbol
for symbol in stock_symbols:
    try:
        # Download historical data for the stock for the specified date range
        data = yf.download(symbol, start=start_date, end=end_date, period='5d')

        # Check if the data is not empty (i.e., download was successful)
        if not data.empty:
            stock_data[symbol] = data
            success_count += 1
            availability.append(1)  # Success, so mark 1
            print(f"Successfully downloaded data for {symbol}")
        else:
            failure_count += 1
            availability.append(0)  # Failure, so mark 0
            print(f"No data available for {symbol}")

    except Exception as e:
        # Handle errors in case the stock symbol is invalid or not found
        failure_count += 1
        availability.append(0)  # Failure, so mark 0
        print(f"Could not download data for {symbol}: {e}")

# Add the 'available' column to stock_name DataFrame
stock_name['available'] = availability

In [ ]:
# Print success and failure counts
print(f"Successfully downloaded data for {success_count} stocks.")
print(f"Failed to download data for {failure_count} stocks.")

In [ ]:
# Assuming stock_name has the columns 'available' and 'sho_pilot'
contingency_table = pd.crosstab(stock_name['available'], stock_name['sho_pilot'])

# Display the contingency table
print(contingency_table)

In [ ]:
# Initialize an empty list to store the sector information
sectors = []

# Loop through each stock symbol
for symbol in stock_symbols:
    try:
        # Download stock information using yfinance
        stock_info = yf.Ticker(symbol)
        
        # Get sector information, or 'Sector not available' if missing
        sector = stock_info.info.get('sector', 'Sector not available')
        sectors.append(sector)
        
        # print(f"Stock: {symbol}, Sector: {sector}")
    
    except Exception as e:
        # In case of an error, mark the sector as 'Sector not available'
        sectors.append('NA')
        # print(f"Could not retrieve sector information for {symbol}: {e}")

# Add the 'sector' column to the stock_name DataFrame
stock_name['sector'] = sectors

# Generate a summary table for the 'sector' variable
summary_table = stock_name['sector'].value_counts()

# Display the summary table
print(summary_table)

In [ ]:
# Generate a summary table for the 'sector' variable
summary_table = stock_name['sector'].value_counts()
# Display the summary table
print(summary_table)

# Filter the stock_name DataFrame to include only available companies
available_stock_name = stock_name[stock_name['available'] == 1]

# Save the 'sho_pilot' and 'sector' columns to a separate CSV file for available companies
available_stock_name[['tsymbol', 'sho_pilot', 'sector']].to_csv('sho_sector_info.csv', index=False)

In [ ]:
# Filter the stock_name DataFrame to get only available companies
available_companies = stock_name[stock_name['available'] == 1]

# Generate a summary table using sector and sho_pilot
summary_table = pd.crosstab(available_companies['sector'], available_companies['sho_pilot'])

# Add a total column to get the sum of companies in each sector
summary_table['Total'] = summary_table.sum(axis=1)

# Sort the table by the 'Total' column in descending order
ranked_table = summary_table.sort_values(by='Total', ascending=False)

# Display the ranked summary table
print(ranked_table)

In [ ]:
# Filter the stock_name DataFrame to get only the companies in a sector
tech_symbols = stock_name[(stock_name['sector'] == 'Energy') & (stock_name['available'] == 1)]['tsymbol'].tolist()

# Initialize an empty list to store the extracted Adj Close data for each symbol
adj_close_data = []

# Loop through each stock symbol in the filtered tech_symbols list
for symbol in tech_symbols:
    if symbol in stock_data:  # Ensure the symbol exists in stock_data
        # Extract the 'Adj Close' column and rename it to the stock symbol
        adj_close_series = stock_data[symbol]['Adj Close'].rename(symbol)
        
        # Append the series to the list
        adj_close_data.append(adj_close_series)

# Combine all the extracted 'Adj Close' series into a single DataFrame, aligning by date
adj_close_df = pd.concat(adj_close_data, axis=1)

# Display the combined DataFrame
print(adj_close_df.head())

# Save the combined Adj Close DataFrame to a CSV file
adj_close_df.to_csv('Energy_data.csv')

In [ ]:
# Filter the stock_name DataFrame to get only the 116 available Technology companies
tech_available = stock_name[(stock_name['sector'] == 'Technology') & (stock_name['available'] == 1)]

# Get the lists of stock symbols for sho_pilot = 1 and sho_pilot = 0
sho_pilot_1_list = tech_available[tech_available['sho_pilot'] == 1]['tsymbol'].tolist()
sho_pilot_0_list = tech_available[tech_available['sho_pilot'] == 0]['tsymbol'].tolist()

# Print the lists
print("Companies with sho_pilot = 1:")
print(sho_pilot_1_list)

print("\nCompanies with sho_pilot = 0:")
print(sho_pilot_0_list)